![UCA](http://caillau.perso.math.cnrs.fr/logo-uca.png)
## L2 MIASHS - ANDE
# TD 3 - Jeux

[![Binder](https://mybinder.org/badge_logo.svg)](https://mybinder.org/v2/gh/jbcaillau/ande/master?urlpath=lab/tree/td3/td3.ipynb)

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/jbcaillau/ande/blob/master/td3/td3.ipynb)

## Exercice 1
On considère le jeu classique de pierre-feuille-ciseaux, à deux joueurs, avec les règles suivantes :
* la pierre casse les ciseaux
* les ciseaux coupent le papier
* le papier recouvre la feuille

Le jeu est simultané, un joueur qui gagne marque un point, un joueur qui ne gagne pas (_i.e._ qui perd ou qui est à égalité) ne marque pas de point.

### 1.1
Mettre le jeu sous forme normale. Existe-t-il des équilibres ?

### 1.2
On suppose désormais que le joueur 2 arrive à deviner quand le joueur 1 s'apprête à jouer ciseaux (et uniquement ce coup). Proposer une nouvelle modélisation, et mettre le jeu sous forme extensive. Existe-t-il des équilibres ? Des stratégies dominées ou dominantes ?

## Exercice 2
On considère le jeu de poker simplifié à deux joueurs suivant. Trois mains sont possibles, numérotées de $1$ à $3$, de la plus faible à la plus forte. Chaque joueuse, ayant reçu l'une de ces mains, met un jeton au pot. Le jeu se déroule ensuite selon le schéma ci-dessous (jeu séquentiel), sachant que, pour _miser_ ou pour _voir_, une joueuse doit rajouter un jeton au pot (rien à mettre au pot si la joueuse choisit de _passer_). La gagnante remporte le pot. 

![Règles du jeu](poker.png)

### 2.1
Définir l'ensemble des stratégies pour chacun des joueuses.

### 2.2
En faisant l'hypothèse que chacune des trois mains à la même chance d'être distribuée, calculer le gain moyen de chaque joueuse pour un couple de stratégies que vous choisirez.

### 2.3
Déterminer la valeur du jeu en stratégies mixtes et proposer des stratégies pour chacune des joueuses.

In [5]:
import numpy as np
from scipy.optimize import linprog

class StrategieInconnue(Exception):
    """Strategie Inconnue
    
    Attribut :
        s -- stratégie
    """
    def __init__(self, s):
        self.s = s

class MainInconnue(Exception):
    """Main Inconnue
    
    Attribut :
        m -- main
    """
    def __init__(self, m):
        self.m = m

class MainVide(Exception):
    """Main Vide
    """

class PbResolution(Exception):
    """Probleme de resolution du LP
    """

S1m = [ 'M', 'PV', 'PP' ] # M = Mise, P = Passe, V = Voit
l1 = len(S1m)
S1 = [ (S1m[i], S1m[j], S1m[k]) for i in range(0, l1) for j in range(0, l1) for k in range(0, l1) ]
m = len(S1)

S2m = [ 'VP', 'VM', 'PP', 'PM' ] # M = Mise, P = Passe, V = Voit
l2 = len(S2m)
S2 = [ (S2m[i], S2m[j], S2m[k]) for i in range(0, l2) for j in range(0, l2) for k in range(0, l2) ]
n = len(S2)

Mains = [ (1, 2), (1, 3), (2, 1), (2, 3), (3, 1), (3, 2) ] # main 1 < main 2 < main 3

def gmfun(s1, s2, main):
    """g = gmfun(s1, s2, main)
    Gain de la joueuse 1 en fonction de la main distribuée aux deux joueuses.

    s1 : triplet de stratégies de la joueuse 1
    s2 : triplet de stratégies de la joueuse 2
    main : main (= cartes) distribuées aux joueuses
    """
    if s1 not in S1: raise StrategieInconnue(s1)
    if s2 not in S2: raise StrategieInconnue(s2)
    if main not in Mains: raise MainInconnue(main)

    s1m = s1[main[0]-1]
    s2m = s2[main[1]-1]

    if s1m[0] is 'M': # la joueuse 1 mise

        if s2m[0] is 'V': # la joueuse 2 voit
            if main[0] > main[1]: gm = 2 # la meilleure main emporte le pot
            else: gm = -2
        elif s2m[0] is 'P': # la joueuse 2 passe
            gm = 1 # la joueuse 1 emporte le pot
        else: raise StrategieInconnue(s2)

    elif s1m[0] is 'P': # la joueuse 1 passe

        if s2m[1] is 'P': # la joueuse 2 passe
            if main[0] > main[1]: gm = 1 # la meilleure main emporte le pot
            else: gm = -1
        elif s2m[1] is 'M': # la joueuse 2 mise
            if s1m[1] is 'V': # la joueuse 1 voit
                if main[0] > main[1]: gm = 2 # la meilleure main emporte le pot
                else: gm = -2
            elif s1m[1] is 'P': # la joueuse 1 passe
                gm = -1
            else: raise StrategieInconnue(s1)
        else: raise StrategieInconnue(s2)

    else: raise StrategieInconnue(s1)

    return gm

def gfun(s1, s2):
    """g = gfun(s1, s2)
    Gain en espérance de la joueuse 1 (toutes les mains sont supposées équiprobables)
 
    s1 : triplet de stratégies de la joueuse 1
    s2 : triplet de stratégies de la joueuse 2
    """
    l = len(Mains)
    if l is 0: raise MainVide
    g = 0.
    for i in range(0, l):
        g = g + (1/l) * gmfun(s1, s2, Mains[i])
    return g

A = np.zeros((m, n))
for i in range(0, m):
    for j in range(0, n):
        A[i, j] = gfun(S1[i], S2[j])

d = abs(np.min(A)) + 1
A = A + d # translate les gains pour garantir une valeur > 0 du jeu mixte

c =-np.ones(n) # min (c|x) = -max (-c|x)
b = np.ones(m) # Ax <= b 
bounds = (0, None) # 0 <= x

method='simplex'
#method='revised simplex'
#method='interior-point'

res = linprog(c, A_ub=A, b_ub=b, bounds=bounds, method=method)
if not res.success: raise PbResolution

v = 1 / (-res.fun)
x = v * res.x
v = v - d
xx = np.round_(x, decimals=8)

print('v = ', v)
print('x = ', xx)
print('')
print("Stratégie mixte optimale de la joueuse 2 :")
for j in range(0, n):
    if xx[j] > 0:
        print(S2[j], "avec proba", xx[j])

v =  -0.055555555555556246
x =  [0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.66666667
 0.         0.         0.         0.         0.         0.
 0.         0.33333333 0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.        ]

Stratégie mixte optimale de la joueuse 2 :
('PP', 'PP', 'VM') avec proba 0.66666667
('PM', 'VP', 'VM') avec proba 0.33333333


## Exercice 3
Alice et Gulliver jouent au jeu suivant : chacun mise d'abord un euro. Alice tire une carte, qui est soit rouge, soit noire, avec même probabilité, et la regarde. Elle choisit alors d'annoncer “rouge” ou “noire”. Pour annoncer “rouge” elle doit miser un euro supplémentaire. Si elle annonce “rouge”, Gulliver peut demander à voir la carte, mais pour cela il doit miser un euro. Si Alice annonce “noire” ou bien si elle annonce “rouge” et que Gulliver demande à voir la carte et observe qu'Alice a menti, Gulliver remporte la mise. Dans les autres cas Alice remporte la mise

### 3.1
Donner la forme extensive du jeu.

*Indications : le hasard est modélisé par un joueur nommé “Nature”. Chaque noeud a pour étiquette le nom du joueur devant décider un choix en ce noeud et les branches partant de ce noeud correspondent au choix que peut faire ce joueur. On relie deux noeuds étiquetés d'un même nom par une ligne pointillée pour indiquer que le joueur ne peut distinguer ces noeuds ; il ne fait donc qu'un seul choix pour cet ensemble de noeuds.*

### 3.2
Donner les formes normales du jeu pour Alice et Gulliver avec comme gains de chaque joueur les espérances de gains. Quelle est l'interprétation de cette espérance de gain lorsque le jeu est répété ?

*Indications : les gains d'Alice et Gulliver dépendent du choix que fait le joueur “Nature”, lequel choisit aléatoirement une carte rouge ou noire de façon équiprobable. Les gains d'Alice et Gulliver sont donc des variables aléatoires dont on retiendra l'espérance.*

### 3.3
Que se passe t-il si Gulliver perçoit chez Alice un signe de réjouissance lorsqu'elle tire une carte rouge ?

### 3.4
Quel est le gain moyen garanti maximal (maxmin) de Gulliver si Alice et Gulliver choisissent leurs stratégies une fois pour toute avant la répétition du jeu ? Si Gulliver choisit une stratégie prudente, le regrètera t-il ? Quel est le gain moyen garanti de Gulliver s'il conteste “rouge” une fois sur deux ?

### 3.5
Gulliver choisit de contester “rouge” aléatoirement avec probabilité $p$. Quelle valeur de $p$ optimise son gain moyen garanti ? Quelle sont les meilleures réponses d'Alice à cette stratégie (dite stratégie mixte prudente) de Gulliver ? Y a t-il parmi ces meilleures réponses une qui fait que Gulliver ne regrette pas son choix ? Comment cela s'interprète-t-il en termes d'équilibre ?

## Exercice 4
Deux marchands de glaces se partagent une clientèle uniformément répartie sur une plage d'un kilomètre en choisissant leurs positions, notées respectivement $x$ et $y$ dans $[0,1]$. Les clients vont au marchand le plus proche. Le gain de chacun des marchands est proportionnel à la part de clientèle captée, de sorte que la somme des gains des deux marchands est indépendante de $x$ et $y$. Si $x = y$, les gains des deux marchands sont égaux.

### 4.1
Quelle est la forme normale du jeu ?

### 4.2
Y a t-il une meilleure réponse du second marchand si le premier choisit $x < 1/2$ ?

### 4.3
Le jeu admet-il un équilibre ? Y a t-il parmi les stratégies du premier marchand des stratégies dominées ?

## Exercice 5
Deux marchands de glaces se partagent une clientèle de cent vacanciers, chaque vacancier achetant exactement une glace. Les marchands ont les mêmes charges : $1$ euro par glace vendue. Ils choisissent indépendamment leur prix de vente : $1$, $2$ ou $3$ euros, et font alors un bénéfice par glace vendue de $0$, $1$ ou $2$ euros. Les vacanciers choisissent tous le marchand le moins cher. Si les deux marchands affichent le même prix, la moitié des vacanciers choisira le premier marchand, l’autre moitié le second.

### 5.1
Mettre le jeu sous forme normale.

### 5.2
Quelle est la meilleure réponse du second marchand si le premier choisit de vendre ses glaces $3$ euros ? Qu’en
est-il si le premier marchand choisit de vendre ses glaces $2$ euros ?

### 5.3
Le jeu possède-t-il des équilibres ?

## Exercice 6
Reprendre l'exercice 3 (menteur) en stratégies mixtes. Quelle est la valeur du jeu pour ce type de stratégies ?

(Source : F.-X. Dehon)